In [1]:
import os
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
import getpass
from typing import Optional
from langchain_core.tools import tool 


LLAMA_CLOUD_API_KEY= "llama api key"
url="url"
api_key="api"
hf_token = "hf"

groq_api = "groq api"

from llama_parse import LlamaParse

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from pydantic import BaseModel, Field
from typing import Optional
import joblib
import nest_asyncio  # noqa: E402
import anyio
nest_asyncio.apply()

d:\workssw\Project_BEAM_llamaParser\llamaparservenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
from llama_index.core import SimpleDirectoryReader
from llama_parse import LlamaParse

parser = LlamaParse(
    api_key=LLAMA_CLOUD_API_KEY,  
    result_type="markdown",  
    verbose=True
)

In [49]:

parser = LlamaParse(
    api_key=LLAMA_CLOUD_API_KEY,  
    result_type="markdown",  
    verbose=True
)

llama_parse_documents = await parser.aload_data("2024-TR_UNIT_PRICES_ENG.pdf",)

Started parsing the file under job_id cac11eca-c065-4c54-b7f9-e8728fb707fc


In [50]:
parsed_doc = llama_parse_documents[0]

In [ ]:
from IPython.display import Markdown
Markdown(parsed_doc.text[:4096])

In [52]:
from pathlib import Path
document_path = Path("parsed_document.md")
with document_path.open("a",encoding="utf-8") as f:
    f.write(parsed_doc.text)

In [3]:
from pathlib import Path
document_path = Path("parsed_document.md")

loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

8407

In [5]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 5065.58it/s]


In [8]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader


url = "http://localhost:6333"
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    url=url,
    prefer_grpc=False,
    collection_name="vector_db"
)

print("Vector DB Successfully Created!")


KeyboardInterrupt: 

In [56]:
### 36 dakika


#qdrant_client = QdrantClient(
    #url="https://424b71be-ca39-4c04-833b-2066313514ea.us-east4-0.gcp.cloud.qdrant.io:6333", 
    #api_key="mmfI5M83sIaW054r1rcdn7qDqw_oeX7Fk9m23PDj8x2qV95lVNf_RA",)

from langchain.vectorstores import Qdrant
import uuid
db_path = f"db_{uuid.uuid4().hex}"

qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path=db_path,
    collection_name="document_embeddings",
)

In [ ]:
chat = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key="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" 
)


In [ ]:
prompt_template = """
You are an experienced architect. Follow the information I have provided meticulously. Do not use any information or datasets other than what is provided.

Your task is to calculate the total cost based on the materials provided, using the information given below.

You will be provided with a list of elements, each with specific attributes. For each element, calculate the total cost based on the following attributes:
- **Element:** The name or type of the element.
- **Type:** The type of the element.
- **Level:** The level or floor where the element is located.
- **Material:** The material of the element.
- **Volume:** The volume of the element in cubic meters (m3).
- **Area:** The area of the element in square meters (m2).
- **Length:** The length of the element in millimeters (mm).

If the type of material is not specified, for example, when asked for 5 m2 of brick, use the average price per m2 of all bricks from the provided information.

Do not provide any additional explanation or response. Your answer should calculate the price of each of the materials entered in terms of total area, volume, or length and give the total cost.

Context: {context}
Question: {question}

The answer should only include the materials you are asked about and their costs. For each material, calculate and indicate the price based on the attributes provided. Ensure that the final output includes the total cost for all elements.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:


from langchain_cohere import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA
retriever = qdrant.as_retriever(search_kwargs={"k": 6})
compressor =  CohereRerank(cohere_api_key="mDzuYHE0Xnq7Jkt0KYGSIJ5iSbslCVIiLHYDQUAT")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)


chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)



In [ ]:
query = """ 
Unreinforced aerated concrete wall block with a thickness of 25 cm, a density of 5.00 N/mm² and a density of 600 kg/m³ (10.130.2543) - 30 m²,
Unreinforced aerated concrete wall block with a thickness of 10 cm and a density of ≥ 2.00 N/mm² and 350 kg/m³ (10.130.2595) - 50 m²,
Unreinforced aerated concrete wall block with a thickness of 15 cm, a density of ≥ 2.00 N/mm² and 350 kg/m³ (10.130.2598) - 20 m²,
Unreinforced aerated concrete wall block with a thickness of 20 cm and a density of ≥ 2.00 N/mm² and 350 kg/m³ (10.130.2601) - 25 m² """

response = chain.invoke(query)
print(response["result"])

UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Wrong input: Vector dimension error: expected dim: 384, got 768"},"time":0.004009459}'

In [ ]:
import pandas as pd

df = pd.read_csv("material_table.csv")
json_data = df.to_json(orient='records')
with open("output.json", "w") as json_file:
    json_file.write(json_data)
query = json.loads(json_data)
response = chain.invoke(query)
print(response["result"])

NameError: name 'pd' is not defined

In [ ]:
import pandas as pd
import json
from collections import defaultdict

df = pd.read_csv("material_table.csv")

required_columns = ["Element", "Type", "Level", "Material", "Volume", "Area", "Length"]
filtered_df = df[required_columns]
element_costs = defaultdict(float)

total_cost = 0

def query_llm(query):
    response = chain.invoke(query)
    return response["result"]

for index, row in filtered_df.iterrows():
    query = {
        "Element": row["Element"],
        "Type": row["Type"],
        "Level": row["Level"],
        "Material": row["Material"],
        "Volume": row["Volume"],
        "Area": row["Area"],
        "Length": row["Length"]
    }
    
    query_json = json.dumps(query)
    
    result = query_llm(query_json)
    
    cost = float(result["cost"])
    
    element_costs[row["Element"]] += cost
    total_cost += cost

for element, cost in element_costs.items():
    print(f"Total cost for {element}: {cost}")

print(f"Total Cost: {total_cost}")